## Hyperparameter tuning for Decision Tree

In [4]:
import pandas as pd
import json
import joblib
import os
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 1. Load Data
df = pd.read_csv('../data/telco_churn_processed.csv')
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Define the "Grid" of settings
param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

# 3. Setup the Grid Search
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           verbose=1,
                           scoring='accuracy')

# 4. Run the Search
print("Starting Hyperparameter Tuning...")
grid_search.fit(X_train, y_train)

# 5. The Results
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"\n✅ Best Parameters Found: {best_params}")
print(f"✅ Best Cross-Validation Accuracy: {best_score:.4f}")

# 6. Evaluate the "Optimized" Model on Test Data
best_model = grid_search.best_estimator_
test_acc = accuracy_score(y_test, best_model.predict(X_test))
print(f"Test Set Accuracy of Optimized Tree: {test_acc:.4f}")

# SAVE THE TUNED MODEL & SCORE
os.makedirs('../models', exist_ok=True)

# Save the Best Model Object
joblib.dump(best_model, '../models/dt_tuned.joblib')
print("✅ Tuned Model saved to '../models/dt_tuned.joblib'")

# Save the Score
dt_tuned_data = {'tuned_dt_accuracy': test_acc, 'best_params': best_params}
with open('../models/dt_tuned_score.json', 'w') as f:
    json.dump(dt_tuned_data, f)
print("✅ Tuned Score saved to '../models/dt_tuned_score.json'")

Starting Hyperparameter Tuning...
Fitting 5 folds for each of 30 candidates, totalling 150 fits

✅ Best Parameters Found: {'criterion': 'entropy', 'max_depth': 7, 'min_samples_split': 2}
✅ Best Cross-Validation Accuracy: 0.7931
Test Set Accuracy of Optimized Tree: 0.7765
✅ Tuned Model saved to '../models/dt_tuned.joblib'
✅ Tuned Score saved to '../models/dt_tuned_score.json'


## 2. Neural Network Hyperparameter Tuning
Unlike Decision Trees, where `GridSearch` is computationally inexpensive, Neural Networks require a **Manual Architecture Search**. We conduct a controlled experiment to determine if increasing model complexity improves performance.

### **Experimental Setup**
* **Model A (Baseline):** The architecture defined in Notebook 03 (Input $\to$ 16 Neurons $\to$ 8 Neurons $\to$ Output). Optimized with `Adam`.
* **Model B (Experiment):** A deeper, wider architecture (Input $\to$ 64 Neurons $\to$ Dropout 20% $\to$ 32 Neurons $\to$ Output). Optimized with `RMSprop`.

### **Hypothesis**
A larger model with Dropout regularization (Model B) should capture more complex non-linear patterns and generalize better than the simpler Model A.

In [5]:
import tensorflow as tf
import json
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout

# 1. Load the Baseline Score
try:
    with open('../models/baseline_score.json', 'r') as f:
        data = json.load(f)
        accuracy_baseline = data['baseline_accuracy']
    print(f"✅ Auto-Loaded Baseline Accuracy: {accuracy_baseline:.4f}")
except FileNotFoundError:
    print("⚠️ Warning: Baseline score file not found. Defaulting to 0.7922")
    accuracy_baseline = 0.7922

# 2. Define Model B
model_b = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 3. Compile with a different optimizer
model_b.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Train Model B
print("Training Neural Network Experiment B...")
history_b = model_b.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

# 5. Evaluate
loss_b, acc_b = model_b.evaluate(X_test, y_test)
print(f"\nModel A (Baseline) Accuracy: {accuracy_baseline:.4f}")
print(f"Model B (Tuned) Accuracy:    {acc_b:.4f}")

# Conclusion Logic
print("-" * 30)
if acc_b > accuracy_baseline:
    print("OBSERVATION: The complex architecture (Model B) improved accuracy.")
    print("RECOMMENDATION: Adopt Model B for deployment.")
else:
    print("OBSERVATION: Increasing complexity (Model B) resulted in similar or lower accuracy.")
    print("CONCLUSION: The simpler Model A is more robust. We recommend proceeding with the Baseline model to reduce computational cost.")
print("-" * 30)

# SAVE THE TUNED MODEL
model_b.save('../models/nn_tuned_model.keras')
print("✅ Tuned Neural Network saved to '../models/nn_tuned_model.keras'")

✅ Auto-Loaded Baseline Accuracy: 0.7922
Training Neural Network Experiment B...
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7964 - loss: 0.4237 

Model A (Baseline) Accuracy: 0.7922
Model B (Tuned) Accuracy:    0.7964
------------------------------
OBSERVATION: The complex architecture (Model B) improved accuracy.
RECOMMENDATION: Adopt Model B for deployment.
------------------------------
✅ Tuned Neural Network saved to '../models/nn_tuned_model.keras'
